In [18]:
# Pytorch Implementation of Group Normalization [CVPR2018 Kaiming He]

import torch
import torch.nn as nn
from torch.nn import Parameter

class GroupNorm(nn.Module):
    def __init__(self, input_channels, group_number=32, eps=1e-5):
        super(GroupNorm, self).__init__()
        self.gamma=Parameter(torch.ones(1, input_channels, 1, 1))
        self.beta=Parameter(torch.zeros(1, input_channels, 1, 1))
        self.group_number=group_number
        self.eps=eps
        
    def forward(self, x):
        N, C, H, W=x.size()
        G=self.group_number
        assert C % G == 0, 'Select a leagal G'
        x1=x.view(N, G, -1)
        x2=x.view(N, G, C//G, H, W)
        
        return x1
        

In [23]:
input_feature=torch.rand((3, 4, 1, 1))
print(input_feature)
groupnorm=GroupNorm(4,2)
output_feature=groupnorm(input_feature)
print(output_feature.shape)
print(output_feature)

tensor([[[[0.7757]],

         [[0.7641]],

         [[0.5792]],

         [[0.8568]]],


        [[[0.5919]],

         [[0.5885]],

         [[0.0883]],

         [[0.4775]]],


        [[[0.9319]],

         [[0.5619]],

         [[0.8672]],

         [[0.2979]]]])
torch.Size([3, 2, 2])
tensor([[[0.7757, 0.7641],
         [0.5792, 0.8568]],

        [[0.5919, 0.5885],
         [0.0883, 0.4775]],

        [[0.9319, 0.5619],
         [0.8672, 0.2979]]])


In [2]:

# Assert expression for debug in python program
b=2
assert b==1, '[{0}] is not equal to 1'.format(b)
print(b)

AssertionError: [2] is not equal to 1

In [3]:
# Assert expression for debug in python program
b=1
assert b==1, '[{0}] is not equal to 1'.format(b)
print(b)

1


In [36]:
x=torch.rand(3,2,2)
var=torch.var(x,1)
print(var)
print(torch.__version__)

tensor([[0.0024, 0.0023],
        [0.0013, 0.0179],
        [0.0796, 0.0072]])
0.4.1


In [32]:
from torch.utils import data
import torch
import torchvision.transforms as T
import os
import cv2
import numpy as np

class MIT5K(data.Dataset):
    def __init__(self, root, train=True, img_size=256, is_transform=False):
        super(MIT5K, self).__init__()
        self.root=root
        self.is_transform=is_transform
        self.img_size=img_size
        self.train=train
        
        if self.train:
            self.subdir='fisheyeshangqi'
        else:
            self.subdir='val'
            
        self.files=os.listdir(os.path.join(self.root, self.subdir+'/image'))
        
    def img_transform(self, img, label):
        img=img.astype(np.float32)
        # BGR 2 RGB
        img=img[:, :, ::-1]
        label=label[:, :, ::-1]
        # After img=cv2.imread(), img is a ndarray with shape [h,w,c], we need [c, h, w]
        img=img.transpose(2, 0, 1)
        label=label.transpose(2, 0, 1)
        # If necessary, img should perform normalization: mean and std
        return img, label
        
    def __len__(self):
        return len(self.files)
                            
    def __getitem__(self, index):
        # Get image path and label path
        base_dir=os.path.join(self.root, self.subdir)
        img_name=self.files[index]
        img_path=base_dir + '/image/' + img_name
        label_path=base_dir + '/label/' + img_name.split('.')[0] + '.png'
        
        # Read image and label
        img=cv2.imread(img_path, cv2.IMREAD_COLOR)
        label=cv2.imread(label_path, cv2.IMREAD_COLOR)
        
        # Resize and Transform
        img=cv2.resize(img, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
        label=cv2.resize(label, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
        img, label=self.img_transform(img, label)
        
        # Convert ndarray to torch.Tensor
        # It's strange that must use img.copy() instead of img, or it will raise a error:
        img=torch.from_numpy(img.copy()).float()
        label=torch.from_numpy(label.copy()).float()
        return img, label
        
        
    

In [33]:
mitdataset=MIT5K('/home/philochan/dataset')
print(len(mitdataset))
mitloader=data.DataLoader(mitdataset, batch_size=16, shuffle=True, num_workers=2)
for i,(img, target) in enumerate(mitloader):
    if i==3:
        break
    print(i, img.shape, target.shape)

400
0 torch.Size([16, 3, 256, 256]) torch.Size([16, 3, 256, 256])
1 torch.Size([16, 3, 256, 256]) torch.Size([16, 3, 256, 256])
2 torch.Size([16, 3, 256, 256]) torch.Size([16, 3, 256, 256])


In [1]:
import os
import cv2
import numpy as np
base_dir=os.path.join('/home/philochan/dataset/fisheyeshangqi', 'image')
print(base_dir)
a=os.listdir(base_dir)
#for ele in a:
#    print(ele.split('.')[0])
print(len(a))

/home/philochan/dataset/fisheyeshangqi/image
400


In [34]:
testimg_path=os.path.join(base_dir,'F000001.jpg')
print(testimg_path)
testimg=cv2.imread(testimg_path)
#print(type(testimg))
#----------------------------------
#testimg=testimg.astype(np.float32)
# BGR 2 RGB
testimg=testimg[:, :, ::-1]
cv2.imshow('img',testimg)
cv2.waitKey(0)

NameError: name 'base_dir' is not defined

In [35]:
def L_reconstruction(inputs, targets, S):
    """ reconstruction loss
    Arguments:
    inputs: [N, 3, H, W]
    targets:[N, 3, H, W]
    S:      [N, 3, H, W]
    """
    pass

In [7]:
import torch.nn.functional as F
def L_color(inputs, targets, S):
    """ color loss
    Arguments:
    inputs:  [N, 3, H, W]
    targets: [N, 3, H, W]
    S:       [N, 3, H, W]
    """
    predictions = 1 / S * inputs
    predictions = F.normalize(predictions, p=2, dim=1)
    targets = F.normalize(targets, p=2, dim=1)
    cosine_value = predictions * targets
    angle = torch.acos(cosine_value)
    return torch.sum(angle)

In [61]:
def L_smoothness(inputs, S):
    """ smoothness loss
    Arguments:
    inputs:  [N, 3, H, W]
    S:       [N, 3, H, W]
    """
    theta=1.2
    epsilon=0.0001
    # torch.log() compute natural logarithm
    Log_images=torch.log(inputs)
    # Define gradient operator
    conv_x = nn.Conv2d(3, 3, (1, 3), padding=(0,1), groups=3, bias=False)
    conv_y = nn.Conv2d(3, 3, (3, 1), padding=(1,0), groups=3, bias=False)
    grad_x_kernel = np.array([[[[-1, 0, 1]]], [[[-1, 0,1]]], [[[-1, 0, 1]]]], dtype='float32')
    conv_x.weight.data = torch.from_numpy(grad_x_kernel)
    grad_y_kernel = np.array([[[[-1], [0], [1]]], [[[-1], [0],[1]]], [[[-1], [0], [1]]]], dtype='float32')
    conv_y.weight.data = torch.from_numpy(grad_y_kernel)
    # Compute gradient image and weights
    Lx = torch.abs(conv_x(Log_images))
    Ly = torch.abs(conv_y(Log_images))
    Wx = 1.0 / (Lx ** theta + epsilon)
    Wy = 1.0 / (Ly ** theta + epsilon)
    # Compute gradient image of illumination map S
    Sx_2 = conv_x(S) ** 2
    Sy_2 = conv_y(S) ** 2
    return torch.sum(Wx * Sx_2 + Wy * Sy_2)

In [62]:
tin1=torch.rand((3,3,8,8))
tin2=torch.rand((3,3,8,8))
lossm=L_smoothness(tin1, tin2)
print(lossm)
print(lossm.shape)

tensor(5703.8359, grad_fn=<SumBackward0>)
torch.Size([])


In [6]:
import torch
t1=torch.rand((5,2,1,1))
#print(t1)
t2= 1 / t1
#print(t2)
t3=t1 * t2
#print(t3)
print(t3[0, :, 0, 0])

tensor([1.0000, 1.0000])


In [10]:
print(torch.log(torch.tensor(10.)))

tensor(2.3026)


In [12]:
import cv2
img_path='/home/philochan/lena.jpeg'
image = cv2.imread(img_path, cv2.IMREAD_COLOR)
laplacian_kernel= torch.tensor([[[0,1,0],[1,-4,1],[0,1,0]], [[0,1,0],[1,-4,1],[0,1,0]], [[0,1,0],[1,-4,1],[0,1,0]]])
print(laplacian_kernel.shape)
output_img=torch.nn.Conv2d()

torch.Size([3, 3, 3])


In [23]:
prefix='https://data.csail.mit.edu/graphics/fivek/img/tiff16_c/'
with open('imagename.txt','r') as f1:
    for line in f1.readlines():
        #用line.strip()把最后的换行符去掉
        image_name=line.strip() + '.tif'
        image_path=prefix + image_name
        with open('download_label.txt', 'a') as f2:
            f2.write(image_path)
            f2.write('\n')

In [58]:
import torch
import numpy as np
import torch.nn as nn
inputs=torch.rand((1, 3, 8, 8))
conv_x = nn.Conv2d(3, 3, (1, 3), padding=(0,1), groups=3, bias=False)
print(conv_x.weight.data.shape)
grad_x_kernel = np.array([[[[-1, 0, 1]]], [[[-1, 0,1]]], [[[-1, 0, 1]]]], dtype='float32')
conv_x.weight.data = torch.from_numpy(grad_x_kernel)

conv_y = nn.Conv2d(3, 3, (3, 1), padding=(1,0), groups=3, bias=False)
grad_y_kernel =torch.from_numpy(np.array([[[[-1], [0], [1]]], [[[-1], [0],[1]]], [[[-1], [0], [1]]]], dtype='float32'))
results = conv_x(inputs)
print(grad_y_kernel.shape)
#print(inputs)
#print(results)
print(conv_y.weight.data.shape)

torch.Size([3, 1, 1, 3])
torch.Size([3, 1, 3, 1])
torch.Size([3, 1, 3, 1])


In [54]:
import torch
torch.__version__

'1.0.0a0+90737f7'

In [67]:
def DoNormalization(x):
    max_num=torch.max(x)
    min_num=torch.min(x)
    return (x - min_num) / (max_num - min_num)

In [68]:
def L_reconstruction(inputs, targets, S):
    inputs=DoNormalization(inputs)
    targets=DoNormalization(targets)
    basic_loss=torch.sum((inputs - S * targets) ** 2)
    OneTensor=torch.ones_like(S)
    penalty_loss=torch.sum(F.relu(inputs-S)) + torch.sum(F.relu(S-OneTensor))
    penalty_param=10000
    return basic_loss + penalty_param * penalty_loss

In [69]:
import torch
import torch.nn.functional as F
tin1=torch.rand((3,3,8,8))
tin3=torch.rand((3,3,8,8))
tin2=torch.rand((3,3,8,8))
lossm=L_reconstruction(tin1, tin2, tin3)
print(lossm)
print(lossm.shape)

tensor(958715.8125)
torch.Size([])


In [70]:
for epoch in range(10):
    print('Epoch: [{}]'.format(epoch))

Epoch: [0]
Epoch: [1]
Epoch: [2]
Epoch: [3]
Epoch: [4]
Epoch: [5]
Epoch: [6]
Epoch: [7]
Epoch: [8]
Epoch: [9]


In [71]:
x=torch.tensor([5.3])
print(x.item())

5.300000190734863


In [ ]:
# Pytorch预训练模型
import torchvision.models as models
# 加载网络结构和预训练参数
resnet18 = models.resnet18(pretrained=True)
# 只加载网络结构，不加载预训练参数
resnet18 = models.resnet18(pretrained=False)
# 打印模型的网络结构
print(resnet18)